In [71]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input

from sklearn.metrics import confusion_matrix



import csv
import keras
import numpy as np
from datetime import datetime
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn

from keras.datasets import mnist
global cm
cm = [0]*10

class NaiveBayes(object):
    def fit(self, X, Y, smoothing=1e+3):
        self.gaussians = dict()
        self.priors = dict()
        labels = set(Y)
        for c in labels:
            current_x = X[Y == c]
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'var': current_x.var(axis=0) + smoothing,
            }
            self.priors[c] = float(len(Y[Y == c])) / len(Y)

    def score(self, X, Y):
        P = self.predict(X)
        for i in range(10):
            cm[i]=0
        for i in range(len(Y)):
          if P[i]!=Y[i]:
            cm[Y[i]]+=1
        return np.mean(P == Y)

    def predict(self, X):
        N, D = X.shape[0], X.shape[1]**2
        K = len(self.gaussians)
        P = np.zeros((N, K))
        for c, g in iteritems(self.gaussians):
            mean, var = g['mean'], g['var']
            P[:,c] = mvn.logpdf(X, mean=mean, cov=var, allow_singular=True) + np.log(self.priors[c])
        return np.argmax(P, axis=1)
    


def data_split(FILE_NAME):
        with open(FILE_NAME, "r") as f:
            data = list(csv.reader(f, delimiter=","))

        data = np.array(data[1:], dtype=np.float32)
        print(data.shape)
        n_samples, n_features = data.shape
        x = data[:, 1:n_features]
        y = data[:, 0]
        return (x,y)
        
        

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy



**Without Smoothing**

In [72]:
if __name__ == '__main__':
    
    
    (Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()
    Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],784))
    Xtest = np.reshape(Xtest,(Xtest.shape[0],784))

    model = NaiveBayes()
    model.fit(Xtrain, Ytrain, smoothing=0)

    train_acc = model.score(Xtrain, Ytrain)
    print("Train accuracy:", train_acc)

    print("Test accuracy:", model.score(Xtest, Ytest))
    print("Digit Wise Error Ratio:")
    s=0
    for i in range(len(cm)):
      s= s+cm[i]

    cm1 = [cm[i]/s for i in range(len(cm))]
    print(cm1)


Train accuracy: 0.6498
Test accuracy: 0.6483
Digit Wise Error Ratio:
[0.026727324424225193, 0.012226329257890247, 0.1612169462610179, 0.10435029854990048, 0.17998294000568665, 0.21580892806369065, 0.020756326414557862, 0.16860961046346318, 0.08757463747512084, 0.022746659084446973]


**With Smoothing**

In [73]:
if __name__ == '__main__':
    
    
    (Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()
    Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],784))
    Xtest = np.reshape(Xtest,(Xtest.shape[0],784))

    model = NaiveBayes()
    model.fit(Xtrain, Ytrain, smoothing=1e+3)
    
    train_acc = model.score(Xtrain, Ytrain)
    print("Train accuracy:", train_acc)

    print("Test accuracy:", model.score(Xtest, Ytest))
    print("Digit Wise Error Ratio:")
    s=0
    for i in range(len(cm)):
      s= s+cm[i]

    cm1 = [cm[i]/s for i in range(len(cm))]
    print(cm1)

Train accuracy: 0.80235
Test accuracy: 0.815
Digit Wise Error Ratio:
[0.04216216216216216, 0.02108108108108108, 0.1372972972972973, 0.10594594594594595, 0.18486486486486486, 0.17189189189189188, 0.05027027027027027, 0.10324324324324324, 0.12486486486486487, 0.05837837837837838]
